# IMPORTS

In [56]:
from collections import *
from functools import *
from itertools import *
import sys
import re
import math
import string
import bisect
import parse
import matplotlib.pyplot as plt
import numpy as np
import heapq
from matplotlib.path import Path
import ast
import hashlib


# SOLVING

In [64]:
compiler = parse.Parser("{country_code}{phone:11.11},")
content = "0085212345678901, +85212345678902, (852)12345678903,"

results = compiler.findall(content)

for result in results:
    print(result)

<Result () {'country_code': '00852', 'phone': '12345678901'}>
<Result () {'country_code': ' +852', 'phone': '12345678902'}>
<Result () {'country_code': ' (852)', 'phone': '12345678903'}>


In [22]:

with open("big.txt", "r") as f:
    data = f.read().splitlines()
    gifts = []
    for i, line in enumerate(data):
        j = line.index(":") + 2
        items = line[j:].split(", ")
        gift = {}
        for item in items:
            name, amt = item.split(": ")
            amt = int(amt)
            gift[name] = amt
        gifts.append(gift)
    target = {"children": 3, "cats": 7, "samoyeds": 2, "pomeranians": 3, "akitas": 0, "vizslas": 0, "goldfish": 5, "trees": 3, "cars": 2, "perfumes": 1}
    for i, partial_gift in enumerate(gifts, start = 1):
        if all(target[key] == partial_gift[key] for key in partial_gift): print(i)


103


In [33]:

with open("big.txt", "r") as f:
    data = f.read().splitlines()
    gifts = []
    for i, line in enumerate(data):
        j = line.index(":") + 2
        items = line[j:].split(", ")
        gift = {}
        for item in items:
            name, amt = item.split(": ")
            amt = int(amt)
            gift[name] = amt
        gifts.append(gift)
    target = {"children": 3, "cats": 7, "samoyeds": 2, "pomeranians": 3, "akitas": 0, "vizslas": 0, "goldfish": 5, "trees": 3, "cars": 2, "perfumes": 1}
    for i, partial_gift in enumerate(gifts, start = 1):
        if (
            all(target[key] < partial_gift.get(key, math.inf) for key in ["cats", "trees"]) and
            all(target[key] > partial_gift.get(key, -math.inf) for key in ["pomeranians", "goldfish"]) and
            all(target[key] == partial_gift.get(key, target[key]) for key in ["children", "samoyeds", "akitas", "vizslas", "cars", "perfumes"])
            ): print("part 2:", i)
        if all(target[key] == partial_gift[key] for key in partial_gift): print("part 1:", i)


part 1: 103
part 2: 405


In [71]:
def bsf(n):
    pos = 0
    while n % 2 == 0:
        pos += 1
        n >>= 1
    return pos
with open("big.txt", "r") as f:
    data = f.read()
    # n = 272 # part 1
    n = 35_651_584
    data = list(map(int, data))
    while len(data) <= n:
        nxt = [c ^ 1 for c in reversed(data)]
        data.append(0)
        data.extend(nxt)
    data = data[:n]
    m = bsf(n)
    delta = 1
    for _ in range(m):
        for i in range(0, len(data), 2 * delta):
            data[i] = 1 if data[i] == data[i + delta] else 0
        delta <<= 1
    print("".join(map(str, [data[i] for i in range(0, n, delta)])))

01010100101011100


In [72]:
from parse import compile
from collections import defaultdict, deque
import time
from typing import List, Dict

def bfs(src: int, dst: int, adj_list: Dict[int, List[int]]) -> int:
    dist = 0
    queue = deque([src])
    vis = set([src])
    while queue:
        sz = len(queue)
        for _ in range(sz):
            node = queue.popleft()
            if node == dst: return dist
            for nei in adj_list[node]:
                if nei in vis: continue
                queue.append(nei)
                vis.add(nei)
        dist += 1
    return -1

def main():
    with open("big.txt", 'r') as f:
        data = f.read().splitlines()
        pat1 = compile("Valve {} has flow rate={:d}; tunnels lead to valves {}")
        pat2 = compile("Valve {} has flow rate={:d}; tunnel leads to valve {}")
        adj_list = defaultdict(list)
        valve_mask = {}
        flow_rates = {}
        for line in data:
            valve, flow, neighbors = pat1.parse(line) if not isinstance(pat1.parse(line), type(None)) else pat2.parse(line)
            neighbors = map(lambda x: x.strip(','), neighbors.split())
            valve_mask[valve] = 1 << len(valve_mask) # key: valve string name, value: integer representation (2^i) 
            flow_rates[valve] = flow
            adj_list[valve].extend(neighbors)
        valves_nonzero_flow = [k for k, v in flow_rates.items() if v]
        minDist = {}
        for src in ['AA'] + valves_nonzero_flow:
            for dst in valves_nonzero_flow:
                if src == dst: continue
                minDist[(src, dst)] = bfs(src, dst, adj_list)
        total_time = 26
        memo = {}
        res = 0
        states = deque([('AA', 0, 0, 0, False)]) # (valve, opened, pressure, time, elephants_turn)
        while states:
            pos, opened, pressure, time, elephants_turn = states.popleft()
            res = max(res, pressure)
            if not elephants_turn:
                states.append(('AA', opened, pressure, 0, True))
            if time == total_time: continue
            for dst in valves_nonzero_flow4
                mask = valve_mask[dst]
                if (opened & mask) != 0: continue
                dist = minDist[(pos, dst)]
                if total_time-dist-1-time < 0: continue
                next_pressure = pressure + flow_rates[dst]*(total_time-dist-1-time)
                key = (mask, opened|mask, time, elephants_turn) # O(n * 2^n * t * num_players)
                if key in memo and next_pressure <= memo[key]: continue
                memo[key] = next_pressure
                states.append((dst, opened|mask, next_pressure, time+dist+1, elephants_turn))
        return res

if __name__ == '__main__':
    start_time = time.perf_counter()
    print(main())
    end_time = time.perf_counter()
    print(f'Time Elapsed: {end_time - start_time:,} seconds')

2351
Time Elapsed: 89.43177149999974 seconds


In [102]:
directions = {
    "E": (0, 1),
    "W": (0, -1),
    "N": (-1, 0),
    "S": (1, 0)
}
with open("big.txt", "r") as f:
    data = f.read().splitlines()
    grid = [list(line) for line in data]
    R, C = len(grid), len(grid[0])
    # init_state = (0, 0, "E")
    init_state = (0, 0, "S")
    vis = set([init_state])
    seen = set([(0, 0)])
    q = deque([init_state])
    in_bounds = lambda r, c: 0 <= r < R and 0 <= c < C
    while q:
        r, c, d = q.popleft()
        seen.add((r, c))
        dr, dc = directions[d]
        r += dr
        c += dc
        if not in_bounds(r, c): continue
        ch = grid[r][c]
        print(r, c, d, ch)
        if ch == "|" and d in "EW":
            if (r, c, "N") not in vis:
                vis.add((r, c, "N"))
                q.append((r, c, "N"))
            if (r, c, "S") not in vis:
                vis.add((r, c, "S"))
                q.append((r, c, "S"))
        elif ch == "-" and d in "NS":
            if (r, c, "E") not in vis:
                vis.add((r, c, "E"))
                q.append((r, c, "E"))
            if (r, c, "W") not in vis:
                vis.add((r, c, "W"))
                q.append((r, c, "W"))
        elif ch == "/":
            if d == "E":
                if (r, c, "N") not in vis:
                    vis.add((r, c, "N"))
                    q.append((r, c, "N"))
            elif d == "N":
                if (r, c, "E") not in vis:
                    vis.add((r, c, "E"))
                    q.append((r, c, "E"))
            elif d == "W":
                if (r, c, "S") not in vis:
                    vis.add((r, c, "S"))
                    q.append((r, c, "S"))
            elif d == "S":
                if (r, c, "W") not in vis:
                    vis.add((r, c, "W"))
                    q.append((r, c, "W"))
        elif ch == "\\":
            if d == "E":
                if (r, c, "S") not in vis:
                    vis.add((r, c, "S"))
                    q.append((r, c, "S"))
            elif d == "N":
                if (r, c, "W") not in vis:
                    vis.add((r, c, "W"))
                    q.append((r, c, "W"))
            elif d == "W":
                if (r, c, "N") not in vis:
                    vis.add((r, c, "N"))
                    q.append((r, c, "N"))
            elif d == "S":
                if (r, c, "E") not in vis:
                    vis.add((r, c, "E"))
                    q.append((r, c, "E"))
        else:
            if (r, c, d) not in vis:
                vis.add((r, c, d))
                q.append((r, c, d))
    # print(vis)
    print(len(seen))




1 0 S .
2 0 S -
2 1 E .
2 2 E .
2 3 E .
2 4 E .
2 5 E -
2 6 E .
2 7 E .
2 8 E .
2 9 E -
2 10 E .
2 11 E .
2 12 E .
2 13 E .
2 14 E .
2 15 E .
2 16 E .
2 17 E .
2 18 E .
2 19 E .
2 20 E .
2 21 E -
2 22 E .
2 23 E .
2 24 E .
2 25 E .
2 26 E .
2 27 E .
2 28 E .
2 29 E .
2 30 E .
2 31 E .
2 32 E .
2 33 E .
2 34 E .
2 35 E .
2 36 E .
2 37 E .
2 38 E |
1 38 N .
3 38 S .
0 38 N .
4 38 S .
5 38 S .
6 38 S .
7 38 S .
8 38 S .
9 38 S .
10 38 S .
11 38 S .
12 38 S .
13 38 S \
13 39 E .
13 40 E .
13 41 E .
13 42 E .
13 43 E .
13 44 E .
13 45 E \
14 45 S .
15 45 S -
15 46 E .
15 44 W -
15 47 E .
15 43 W .
15 48 E .
15 42 W .
15 49 E .
15 41 W .
15 50 E .
15 40 W .
15 51 E .
15 39 W .
15 52 E .
15 38 W .
15 53 E .
15 37 W .
15 54 E .
15 36 W .
15 55 E .
15 35 W .
15 56 E .
15 34 W .
15 57 E .
15 33 W .
15 58 E .
15 32 W .
15 59 E .
15 31 W .
15 60 E .
15 30 W .
15 61 E .
15 29 W .
15 62 E .
15 28 W .
15 63 E -
15 27 W .
15 64 E .
15 26 W .
15 65 E .
15 25 W .
15 66 E .
15 24 W /
15 67 E .
16 24 S .


In [151]:
move = {
    "E": 1,
    "W": -1,
    "N": -1j,
    "S": 1j
}
rotations = {
    1: {
        "/": [-1j],
        "\\": [1j],
        "|": [1j, -1j],
        "-": [1],
        ".": [1]
    },
    -1: {
        "/": [-1j],
        "\\": [1j],
        "|": [1j, -1j],
        "-": [1],
        ".": [1]
    },
    1j: {
        "/": [1j],
        "\\": [-1j],
        "|": [1],
        "-": [1j, -1j],
        ".": [1]
    },
    -1j: {
        "/": [1j],
        "\\": [-1j],
        "|": [1],
        "-": [1j, -1j],
        ".": [1]
    }
}

with open("big.txt", "r") as f:
    data = f.read().splitlines()
    grid = [list(line) for line in data]
    R, C = len(grid), len(grid[0])
    columns = {
        0: "E",
        C - 1: "W"
    }
    rows = {
        0: "S",
        R - 1: "N"
    }
    def bfs(q, memo):
        seen = set()
        in_bounds = lambda r, c: 0 <= r < R and 0 <= c < C
        while q:
            z, d = q.popleft()
            seen.add(z)
            z += d
            r, c = map(int, (z.imag, z.real))
            if not in_bounds(r, c): continue
            rots = rotations[d][grid[r][c]]
            for rot in rots:
                if (z, d * rot) in memo: continue
                memo.add((z, d * rot))
                q.append((z, d * rot))
        return len(seen)
    res = 0
    starts = [(0, c) for c in range(C)] + [(R - 1, c) for c in range(C)] + [(r, 0) for r in range(1, R - 1)] + [(r, C - 1) for r in range(1, R - 1)]
    for r, c in starts:
        z = c + r * j
        if r in rows:
            q = deque()
            memo = set()
            d = move[rows[r]]
            rots = rotations[d][grid[r][c]]
            for rot in rots:
                q.append((z, d * rot))
                memo.add((z, d * rot))
            current = bfs(q, memo)
            res = max(res, current)
        if c in columns:
            q = deque()
            memo = set()
            d = move[columns[c]]
            rots = rotations[d][grid[r][c]]
            for rot in rots:
                q.append((z, d * rot))
                memo.add((z, d * rot))
            current = bfs(q, memo)
            if z == 0:
                print("part 1:", current)
            res = max(res, current)
    print("part 2:", res)


part 1: 7477
part 2: 7557


In [47]:
directions = {
    "U": (-1, 0),
    "D": (1, 0),
    "L": (0, -1),
    "R": (0, 1)
}
dirs = "UDLR"
open = "bcdef"
with open("big.txt", "r") as f:
    grid = [list(line) for line in f.read().splitlines()]
    code = "ihgpwlah"
    R, C = len(grid), len(grid[0])
    sr = sc = None
    for r, c in product(range(R), range(C)):
        if grid[r][c] == "S":
            sr, sc = r, c
            break
    seen = set([(sr, sc, code)])
    q = deque([(sr, sc, code)])
    while q:
        r, c, path = q.popleft()
        h = hashlib.md5(path.encode("utf-8")).hexdigest()[:4]
        for i, d in enumerate(dirs):
            if h[i] not in open: continue # locked
            path += dirs[i]
        break

de9f


In [96]:
directions = {
    "E": (0, 1),
    "W": (0, -1),
    "N": (-1, 0),
    "S": (1, 0)
}
with open("big.txt", "r") as f:
    data = f.read().splitlines()
    grid = [list(map(int, line)) for line in data]
    R, C = len(grid), len(grid[0])
    in_bounds = lambda r, c: 0 <= r < R and 0 <= c < C
    memo = defaultdict(lambda: math.inf)
    heap = [(0, 0, 0, 0, "E")]
    while heap:
        cost, r, c, dist, d = heapq.heappop(heap)
        if (r, c) == (R - 1, C - 1): print(cost)
        dr, dc = directions[d]
        nr, nc = r + dr, c + dc
        if not in_bounds(nr, nc): continue
        ncost = cost + grid[nr][nc]
        if ncost < memo[(nr, nc, dist, d)]: 
            memo[(nr, nc, dist, d)] = ncost
            if dist < 2: # go straight
                heapq.heappush(heap, (ncost, nr, nc, dist + 1, d))
            if d in "EW":
                heapq.heappush(heap, (ncost, nr, nc, 0, "N"))
                heapq.heappush(heap, (ncost, nr, nc, 0, "S"))
            else:
                heapq.heappush(heap, (ncost, nr, nc, 0, "E"))
                heapq.heappush(heap, (ncost, nr, nc, 0, "W"))

# for r, c in path:
#     grid[r][c] = "#"

# for line in grid:
#     print("".join(map(str, line)))

1040
1040
1040
1041
1041
1041
1041
1041
1042
1042
1042
1043
1043
1043
1043
1043


In [ ]:
"""
1040
1041
1042
1043

"""

In [13]:
# f = g + h, cost + heuristic
with open("big.txt", "r") as f:
    data = f.read().splitlines()
    grid = [list(map(int, line)) for line in data]
    R, C = len(grid), len(grid[0])
    in_bounds = lambda r, c: 0 <= r < R and 0 <= c < C
    memo = set()
    heap = [(R + C, 0, 0, 0, 0, 0, 3)]
    def g(cost, r, c):
        return cost + grid[r][c]
    while heap:
        f, cost, r, c, dr, dc, steps = heapq.heappop(heap)
        if (r, c, dr, dc, steps) in memo: continue
        memo.add((r, c, dr, dc, steps))
        if (r, c) == (R - 1, C - 1): print(cost); break
        if steps < 3:
            nr, nc = r + dr, c + dc
            if in_bounds(nr, nc):
                heapq.heappush(heap, (R + C - nr - nc + g(cost, nr, nc), g(cost, nr, nc), nr, nc, dr, dc, steps + 1))
        for ndr, ndc in [(0, 1), (1, 0), (-1, 0), (0, -1)]:
            if (ndr, ndc) == (dr, dc) or (ndr, ndc) == (-dr, -dc): continue
            nr, nc = r + ndr, c + ndc
            if in_bounds(nr, nc):
                heapq.heappush(heap, (R + C - nr - nc + g(cost, nr, nc), g(cost, nr, nc), nr, nc, ndr, ndc, 1))

1039


In [112]:
with open("big.txt", "r") as f:
    data = f.read().splitlines()
    grid = [list(map(int, line)) for line in data]
    R, C = len(grid), len(grid[0])
    in_bounds = lambda r, c: 0 <= r < R and 0 <= c < C
    memo = set()
    heap = [(0, 0, 0, 0, 0, 10)]
    while heap:
        cost, r, c, dr, dc, steps = heapq.heappop(heap)
        if (r, c, dr, dc, steps) in memo: continue
        memo.add((r, c, dr, dc, steps))
        if steps >= 4 and (r, c) == (R - 1, C - 1): print(cost); break
        if steps < 10: # cannot go straight after 10 steps
            nr, nc = r + dr, c + dc
            if in_bounds(nr, nc):
                heapq.heappush(heap, (cost + grid[nr][nc], nr, nc, dr, dc, steps + 1))
        if steps >= 4: # minimum of 4 steps before turns
            for ndr, ndc in [(0, 1), (1, 0), (-1, 0), (0, -1)]:
                if (ndr, ndc) == (dr, dc) or (ndr, ndc) == (-dr, -dc): continue
                nr, nc = r + ndr, c + ndc
                if in_bounds(nr, nc):
                    heapq.heappush(heap, (cost + grid[nr][nc], nr, nc, ndr, ndc, 1))

1201


In [ ]:
turn_on = compile("turn on {:d},{:d} through {:d},{:d}")
turn_off = compile("turn off {:d},{:d} through {:d},{:d}")
toggle = compile("toggle {:d},{:d} through {:d},{:d}")
with open('input.txt', 'r') as f:
    data = f.read().splitlines()
    lights = [[0] * 1000 for _ in range(1000)]
    for line in data:
        to, toff, tog = turn_on.parse(line), turn_off.parse(line), toggle.parse(line)
        if to is not None:
            r1, c1, r2, c2 = to.fixed
            for r, c in product(range(r1, r2 + 1), range(c1, c2 + 1)):
                lights[r][c] += 1
        elif toff is not None:
            r1, c1, r2, c2 = toff.fixed
            for r, c in product(range(r1, r2 + 1), range(c1, c2 + 1)):
                lights[r][c] = max(0, lights[r][c] - 1)
        else:
            r1, c1, r2, c2 = tog.fixed
            for r, c in product(range(r1, r2 + 1), range(c1, c2 + 1)):
                lights[r][c] += 2
    print(sum(map(sum, lights)))